In [1]:
from mxnet import gluon, nd
from mxnet.gluon import nn

class CenterLayer(nn.Block):
    def __init__(self,**kwargs):
        super(CenterLayer, self).__init__(**kwargs)
        
    def forward(self, x):
        return x - x.mean()
    

In [2]:
#不含模型参数
layer = CenterLayer()
layer(nd.array([1,2,3,4,5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

In [3]:
net = nn.Sequential()
net.add(nn.Dense(128),CenterLayer())
net.initialize()
y = net(nd.random.uniform(shape=(4,8)))
y.mean().asscalar()

-7.212293e-10

In [4]:
#自定义模型参数，params.get()函数
params = gluon.ParameterDict()
params.get('param2',shape=(2,3))
params


(
  Parameter param2 (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

In [12]:
#定义一个含weight和bias的全连接层，units代表输出，in_units代表输入
class MyDense(nn.Block):
    def __init__(self,units,in_units,**kwargs):
        super(MyDense,self).__init__(**kwargs)  #继承Block的属性
        self.weight = self.params.get('weight',shape=(in_units,units))
        self.bias = self.params.get('bias',shape=(units,))
    def forward(self,x):
        linear = nd.dot(x,self.weight.data())+self.bias.data()
        return nd.relu(linear)

In [13]:
#实例化
dense = MyDense(units = 3,in_units=5)
dense.params

mydense2_ (
  Parameter mydense2_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense2_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

In [14]:
#前向计算
dense.initialize()
dense(nd.random.uniform(shape=(2,5)))


[[0.         0.12363369 0.        ]
 [0.         0.17685556 0.        ]]
<NDArray 2x3 @cpu(0)>

In [15]:
#自定义层
net = nn.Sequential()
net.add(MyDense(8,in_units=64),MyDense(1,in_units=8))
net.initialize()
net(nd.random.uniform(shape=(2,64)))


[[0.00676564]
 [0.01276375]]
<NDArray 2x1 @cpu(0)>